## Merge

https://www.kaggle.com/uciml/restaurant-data-with-consumer-ratings

In [1]:
import pandas as pd

In [2]:
rating_df = pd.read_csv('./data/cuisine/rating_final.csv')

In [3]:
rating_df

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2
...,...,...,...,...,...
1156,U1043,132630,1,1,1
1157,U1011,132715,1,1,0
1158,U1068,132733,1,1,0
1159,U1068,132594,1,1,1


In [4]:
cuisine_df = pd.read_csv('./data/cuisine/chefmozcuisine.csv')
cuisine_df

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food
...,...,...
911,132005,Seafood
912,132004,Seafood
913,132003,International
914,132002,Seafood


In [5]:
merged_df = pd.merge(rating_df, cuisine_df, on='placeID', how='inner')
merged_df

,userID,placeID,rating,food_rating,service_rating,Rcuisine
0,U1077,135085,2,2,2,Fast_Food
1,U1108,135085,1,2,1,Fast_Food
2,U1081,135085,1,2,1,Fast_Food
3,U1056,135085,2,2,2,Fast_Food
4,U1134,135085,2,1,2,Fast_Food
...,...,...,...,...,...,...
1038,U1061,132958,2,2,2,American
1039,U1025,132958,1,0,0,American
1040,U1097,132958,2,1,1,American
1041,U1096,132958,1,2,2,American


In [6]:
merged_df[merged_df.Rcuisine.isna()]

,userID,placeID,rating,food_rating,service_rating,Rcuisine


In [7]:
merged_df.Rcuisine.unique()

array(['Fast_Food', 'Mexican', 'Seafood', 'Pizzeria', 'Regional',
       'Armenian', 'Cafeteria', 'American', 'Bar', 'Bar_Pub_Brewery',
       'Italian', 'Japanese', 'Contemporary', 'Burgers', 'Mediterranean',
       'International', 'Vietnamese', 'Family', 'Chinese',
       'Cafe-Coffee_Shop', 'Game', 'Bakery', 'Breakfast-Brunch'],
      dtype=object)

In [8]:
merged_df[merged_df.Rcuisine=='Bar_Pub_Brewery'][['rating', 'food_rating', 'service_rating']].mean()

rating            1.305085
food_rating       1.169492
service_rating    1.203390
dtype: float64

In [9]:
merged_df[merged_df.Rcuisine=='Bar'][['rating', 'food_rating', 'service_rating']].mean()

rating            1.200000
food_rating       1.135714
service_rating    1.085714
dtype: float64

In [10]:
merged_df[merged_df.Rcuisine=='Cafeteria'][['rating', 'food_rating', 'service_rating']].mean()

rating            1.205882
food_rating       1.127451
service_rating    1.078431
dtype: float64

In [11]:
merged_df[merged_df.Rcuisine=='Cafe-Coffee_Shop'][['rating', 'food_rating', 'service_rating']].mean()

rating            1.583333
food_rating       1.333333
service_rating    1.416667
dtype: float64

In [12]:
user_payment_df = pd.read_csv('./data/cuisine/userpayment.csv')

In [13]:
payment_df = pd.merge(rating_df, user_payment_df, how='inner')

In [14]:
payment_df.Upayment.unique()

array(['VISA', 'cash', 'bank_debit_cards', 'MasterCard-Eurocard',
       'American_Express'], dtype=object)

In [15]:
payment_df[payment_df.Upayment=='cash'][['rating', 'food_rating', 'service_rating']].mean()

rating            1.182815
food_rating       1.200183
service_rating    1.080439
dtype: float64

In [16]:
payment_df[payment_df.Upayment=='bank_debit_cards'][['rating', 'food_rating', 'service_rating']].mean()

rating            1.437908
food_rating       1.562092
service_rating    1.398693
dtype: float64